In [3]:
import sys
sys.path.append('../')

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pandas as pd
from pygbif import species
from pygbif import occurrences as occ
from geopy.geocoders import Nominatim

import src.get_species as gs

In [8]:
sp_list = ["Fucus serratus", "Ascophyllum nodosum", "Pelvetia canaliculata", "Bifurcaria bifurcata",
           "Ulva lactuca", "Fucus vesiculosus", "Fucus spiralis", "Codium tomentosum", "Sargassum muticum", "Laminaria hyperborea", 
          "Palmaria palmata", "Alaria esculenta", "Laminaria digitata", "Himanthalia elongata", "Halidrys siliquosa"
          "Saccharina latissima", "Undaria pinnatifida", "Codium fragile", "Gratelupia turuturu"]

In [9]:
sp, codes = gs.get_species_name_from_codes(sp_list)

In [10]:
codes

{'Fucus serratus': 3196437,
 'Ascophyllum nodosum': 3196524,
 'Pelvetia canaliculata': 3196499,
 'Bifurcaria bifurcata': 3197821,
 'Ulva lactuca': 5273309,
 'Fucus vesiculosus': 8222574,
 'Fucus spiralis': 3196369,
 'Codium tomentosum': 5272167,
 'Sargassum muticum': 3197314,
 'Laminaria hyperborea': 5422383,
 'Palmaria palmata': 7843860,
 'Alaria esculenta': 5422592,
 'Laminaria digitata': 5422479,
 'Himanthalia elongata': 3197836,
 'Halidrys siliquosaSaccharina latissima': 8082072,
 'Undaria pinnatifida': 5422556,
 'Codium fragile': 5272096,
 'Gratelupia turuturu': 2658889}

In [11]:
sp

[3196437,
 3196524,
 3196499,
 3197821,
 5273309,
 8222574,
 3196369,
 5272167,
 3197314,
 5422383,
 7843860,
 5422592,
 5422479,
 3197836,
 8082072,
 5422556,
 5272096,
 2658889]

In [19]:
%%time
results = gs.get_coordinates(sp)

CPU times: user 6.16 s, sys: 2.06 s, total: 8.22 s
Wall time: 2min 51s


In [20]:
results.to_csv("macroalgae_initial.csv", index = False)

In [21]:
results = pd.read_csv("macroalgae_initial.csv")

In [22]:
results.head()

,lon,lat,locality,year,month,kingdom,class,family,genus,species
0,1.60648,50.77092,NaN,1900,1.0,Chromista,Phaeophyceae,Laminariaceae,Laminaria,Laminaria flexicaulis Le Jol.
1,1.61021,50.76952,NaN,1900,1.0,Chromista,Phaeophyceae,Laminariaceae,Laminaria,Laminaria flexicaulis Le Jol.
2,NaN,NaN,NaN,1900,1.0,Plantae,Ulvophyceae,Ulvaceae,Ulva,Ulva fasciata Delile
3,NaN,NaN,Dorchester,1900,1.0,Chromista,Phaeophyceae,Fucaceae,Fucus,Fucus vesiculosus L.
4,NaN,NaN,"Waialua, oahu",1900,1.0,Plantae,Ulvophyceae,Ulvaceae,Ulva,Ulva fasciata Delile


In [23]:
results = gs.clean(results)

In [24]:
results = gs.divide_species(results, "species")

In [25]:
results.head()

,lon,lat,locality,year,month,kingdom,class,family,genus,species
0,1.6065,50.7709,NaN,1900,1.0,Chromista,Phaeophyceae,Laminariaceae,Laminaria,Laminaria flexicaulis
1,1.6102,50.7695,NaN,1900,1.0,Chromista,Phaeophyceae,Laminariaceae,Laminaria,Laminaria flexicaulis
6,-158.1254,21.5714,"Waialua, Oahu, HI",1900,1.0,Plantae,Ulvophyceae,Codiaceae,Codium,Codium tomentosum
7,-158.1254,21.5714,"Waialua, Oahu, HI",1900,1.0,Plantae,Ulvophyceae,Ulvaceae,Ulva,Ulva fasciata
21,-158.1830,21.4333,Waianae,1900,5.0,Plantae,Ulvophyceae,Ulvaceae,Ulva,Ulva lactuca


In [26]:
results.species.unique()

array(['Laminaria flexicaulis', 'Codium tomentosum', 'Ulva fasciata',
       'Ulva lactuca', 'Pelvetia canaliculata', 'Fucus vesiculosus',
       'Palmaria palmata', 'Laminaria stenophylla', 'Alaria esculenta',
       'Ascophyllum nodosum', 'Fucus spiralis', 'Himanthalia elongata',
       'Laminaria digitata', 'Halidrys siliquosa', 'Fucus serratus',
       'Laminaria hyperborea', 'Rhodymenia palmata', 'Fucus platycarpus',
       'Alaria musaefolia', 'Fucus canaliculatus', 'Undaria pinnatifida',
       'Codium fragile', 'Laminaria intermedia', 'Palmaria mollis',
       'Bifurcaria bifurcata', 'Fucus inflatus', 'Alaria grandifolia',
       'Halidrys dioica', 'Alaria dolichorhachis', 'Halidrys Lyngbye,',
       'Fucus rotundatus', 'Ascophyllum mackaii', 'Grateloupia turuturu',
       'Sargassum muticum', 'Fucus nodosus', 'Fucus mytili',
       'Codium mucronatum', 'BOLD:AAD7070', 'BOLD:AAB0883'], dtype=object)

Laminaria flexicaulis ==> eliminarla

Ulva fasciata ==> Ulva lactuca 

Laminaria stenophylla ==> Laminaria digitata

Rhodymenia palmata ==> Palmaria palmata

Fucus platycarpus ==> Fucus spiralis

Alaria musaefolia == > Alaria esculenta

Fucus canaliculatus ==> Fucus vesiculosus

Laminaria intermedia ==> Laminaria digitata

Palmaria mollis ==> eliminar

Fucus inflatus ==> Fucus vesiculosus

Alaria grandifolia ==> Alaria esculenta

Halidrys dioica ==> eliminarla

Alaria dolichorhachis ==> Alaria esculenta

Halidrys Lyngbye ==> eliminarla

Fucus rotundatus ==> eliminarla

Ascophyllum mackaii ==> eliminarla

Fucus nodosus ==> Ascophyllum nodosum

Fucus mytili == Fucus vesiculosus

BOLD:AAB0883 ==> eliminar

In [27]:
final = gs.clean_species(results, "species")

In [ ]:
final["new"] = final["lat"].map(str) + "," + final["lon"].map(str)
final["locality"] = final.new.apply(gs.get_community)

In [ ]:
final.head()

In [ ]:
final2 = final[final["locality"] != "unknown"]

In [ ]:
final2.head()

In [ ]:
final2[['locality', 'state', "country"]] = pd.DataFrame(final2['locality'].tolist(), index=final2.index)

In [ ]:
final2.head()

In [ ]:
final2.to_csv("macroalgae_final.csv")